In [1]:
import numpy as np
import torch
import torch.nn as nn

In [33]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

rnn = RNN(5, 32, 15)

In [34]:
learning_rate = 0.005
criterion = nn.NLLLoss()

def train(y_train, x_train):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(x_train.size()[0]):
        output, hidden = rnn(x_train[i], hidden)

    loss = criterion(output, y_train)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [44]:
test_x = torch.rand(1000,1,5)
test_y = torch.tensor([14] ,dtype=torch.long)

train(test_y, test_x)


(tensor([[-2.6934, -2.9092, -2.6919, -2.6914, -2.5659, -2.9095, -2.3219, -2.9265,
          -3.0863, -2.6758, -2.9910, -2.6195, -2.4216, -2.5889, -2.8508]],
        grad_fn=<LogSoftmaxBackward>), 2.8508291244506836)